<a href="https://colab.research.google.com/github/dhuanca/deeplearning/blob/main/utils.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Instalaciones

In [ ]:
!python -m pip install statsmodels

In [ ]:
# Configuraciones de acceso

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Importación de librerias

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
# Importando de keras las librerias mas importantes!
from keras.models import Sequential # Arquitectura de red neuronal!
from keras.layers import Dense      # Capa densa!
from keras.layers import LSTM       # Capa recurrente
from keras.layers import Dropout    # Evita el overfitting (Inactiva algunas neuronas)
from sklearn.metrics import mean_squared_error

In [ ]:
# Creacion de objeto datasetIngestado con atributos de los distintos dataframes del proceso

In [ ]:
class datasetIngestado():
  def __init__(self, ruta):
    self.df = pd.read_csv(ruta,
                parse_dates=['Date'],
                dayfirst=True,
                index_col='Date')
    self.df.sort_values('Date', inplace=True, ascending=True)
    self.train = pd.DataFrame()
    self.test = pd.DataFrame()
    self.model = Sequential()
    self.modelo_entrenado = 1
    self.train_max = 0.0
    self.train_min = 0.0

  def imprimir_dataset(self, n_filas):
    print(self.df.head(n_filas))

  def imprime_forma(self):
    print(self.df.shape)

  def visualiza_dataset(self, variable):
    plt.figure(num=None, figsize=(15, 6), dpi=80, facecolor='w', edgecolor='k')
    self.df[variable].plot()
    plt.tight_layout()
    plt.grid()
    plt.show()

  def descompone_serie(self, variable, n_periodos):
    res = sm.tsa.seasonal_decompose(self.df[variable],period=n_periodos)
    fig = res.plot()
    fig.set_figheight(8)
    fig.set_figwidth(15)
    plt.show()

  def descripcion_dataset(self):
    print(self.df.describe())

  def divide_train_test(self, variable, cantidad):
    self.train, self.test = self.df[[variable]].iloc[0:-cantidad], self.df[[variable]].iloc[-cantidad:len(self.df)]
    return self.train, self.test

  """
   Normalizar los data sets
   Duda: para el escalado, el dataset de test tambien se escala con los min max de entrenamiento?
  """
  def escala_train_test(self, data_train, data_test):
    self.train_max = data_train.max()
    self.train_min = data_train.min()
    train_set_scaled = (data_train - self.train_min)/(self.train_max - self.train_min)
    test_set_scaled = (data_test - self.train_min)/(self.train_max - self.train_min)
    return train_set_scaled, test_set_scaled

  """
    Define arquitectura
  """
  def create_dataset(self, X, y, v_time_steps):
    Xs, ys = [], []
    for i in range(len(X) - v_time_steps):
        v = X.iloc[i:(i + v_time_steps)].values
        Xs.append(v)
        ys.append(y.iloc[i + v_time_steps])
    return np.array(Xs), np.array(ys)

  def crea_datasets_train_test(self, data_escalada, time_steps):
    X_data, y_data = self.create_dataset(data_escalada, data_escalada, time_steps)
    return  X_data, y_data

  def create_dataset_with_preds(self, X, y, pred_steps, v_time_steps):
    Xs, ys = [], []
    for i in range(len(X) - v_time_steps - pred_steps):
        v = X.iloc[i:(i + v_time_steps)].values
        Xs.append(v)
        ys.append(y.iloc[i + v_time_steps + pred_steps - 1])
    return np.array(Xs), np.array(ys)

  def crea_datasets_train_test_preds(self, data_escalada, pred_steps, time_steps):
    X_data, y_data = self.create_dataset_with_preds(data_escalada, data_escalada, pred_steps, time_steps)
    return  X_data, y_data

  def lstm_architecture(self, X_data,rate_dropout):
    # Inicializando the RNN
    # self.model = Sequential()

    # 1ra capa LSTM y Dropout para regularización.
    # input_shape (amplitude,1)
    self.model.add(LSTM(units = 250, return_sequences = True, input_shape=(X_data.shape[1], X_data.shape[2])))
    # 20% de las neuronas serán ignoradas durante el training (20%xNodos = 10)
    # Para hacer menos probable el overfiting
    self.model.add(Dropout(rate=rate_dropout))

    # 2da capa LSTM y Dropout para regularización.
    self.model.add(LSTM(units = 250, return_sequences = True))
    self.model.add(Dropout(rate=rate_dropout))

    # 3ra capa LSTM y Dropout para regularización.
    self.model.add(LSTM(units = 250, return_sequences = True))
    self.model.add(Dropout(rate=rate_dropout))

    # 4ta capa LSTM y Dropout para regularización.
    self.model.add(LSTM(units = 250, return_sequences = False))
    self.model.add(Dropout(rate=rate_dropout))

    # Capa de Salida!
    self.model.add(Dense(units = 1))

    # Resumen del modelo!
    self.model.summary()

    # return self.model


  def compila_red_reuronal(self, X_train, rate_dropout, optimizador, perdida):
    self.lstm_architecture(X_train,rate_dropout)
    self.model.compile(optimizer = optimizador, loss = perdida)

  def ejecuta_red_reuronal(self, X_train, y_train, v_epochs, v_batch_size, v_shuffle):
    self.modelo_entrenado = self.model.fit(X_train,
                                          y_train,
                                          epochs=v_epochs,
                                          batch_size=v_batch_size,
                                          shuffle=v_shuffle)

  def grafica_de_perdida(self):
    plt.plot(self.modelo_entrenado.history['loss'], label='train')
    plt.legend()
    plt.show()

  def prediccion(self, X_test):
    y_pred = self.model.predict(X_test)
    return y_pred

# Regresamos la informacion a los valores originales!
  def valores_originales(self, y_test_esca, y_pred_esca, y_train_esca):
    y_test = y_test_esca*(self.train_max[0] - self.train_min[0]) + self.train_min[0]    # 100 valores reales de test!
    y_pred = y_pred_esca*(self.train_max[0] - self.train_min[0]) + self.train_min[0]    # 100 valores pronosticados para validar!
    y_train = y_train_esca*(self.train_max[0] - self.train_min[0]) + self.train_min[0]  # 1732 valores de entrenamiento!
    return y_test, y_pred, y_train

  def grafica_predicciones(self, y_test, y_pred, y_train ):
    plt.figure(num=None, figsize=(15, 6), dpi=80, facecolor='w', edgecolor='k')
    plt.plot(np.arange(len(y_train), len(y_train) + len(y_test)), y_test.flatten(), marker='.', label="true")
    plt.plot(np.arange(len(y_train), len(y_train) + len(y_test)), y_pred.flatten(), 'r', marker='.', label="prediction")
    plt.plot(np.arange(0, len(y_train)), y_train.flatten(), 'g', marker='.', label="history")
    plt.ylabel('Count')
    plt.xlabel('Time Step')
    plt.legend()
    plt.show()


  def evaluacion_modelo(self, y_test, y_pred):
    # Vemos algunos indicadores del ajuste!
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    print(f'RMSE: ',rmse)
    # Definimos y calculamos el MAPE (mean_absolute_percentage_error)
    y_test, y_pred = np.array(y_test), np.array(y_pred)
    mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100
    print(f'MAPE: ',mape)
    return rmse, mape

In [ ]:
"""
   Variables:
   variable - Columna del dataset a usar en la serie de tiempo
   n_registros_test - número de registros contando desde el final de la serie a ser tomados como datos de test
   time_steps - lag de tiempo a predecir
   dropout - Porcentaje expresado en decimales para la cantidad de dropout de neuronas
   optimizador - metodo de optimización 'adam'
   perdida - metrica para controlar la perdida de gradiente
   n_epocs - cantidad de iteraciones
   batch  - tamaño del lote de cada iteracion
   v_shuffle  - si shuffle sera verdadero o falso

"""

def genera_modelo(variable, n_registros_test, time_steps, dropout, optimizador, perdida, n_epocs, batch, v_shuffle):
  df_train, df_test = df.divide_train_test(variable, n_registros_test)
  df_train_escalado, df_test_escalado = df.escala_train_test(df_train, df_test)
  X_data_train, y_data_train = df.crea_datasets_train_test(df_train_escalado,time_steps)
  X_data_test, y_data_test = df.crea_datasets_train_test(df_test_escalado,time_steps)
  df.compila_red_reuronal(X_data_train, dropout, optimizador, perdida)
  df.ejecuta_red_reuronal(X_data_train, y_data_train, n_epocs, batch, v_shuffle)
  df.grafica_de_perdida()
  y_predicho = df.prediccion(X_data_test)
  y_test_orig, y_pred_orig, y_train_orig = df.valores_originales(y_data_test, y_predicho, y_data_train)
  df.grafica_predicciones(y_test_orig, y_pred_orig, y_train_orig)
  return df.evaluacion_modelo(y_test_orig, y_pred_orig)


In [ ]:
"""
   Variables:
   variable - Columna del dataset a usar en la serie de tiempo
   n_registros_test - número de registros contando desde el final de la serie a ser tomados como datos de test
   time_steps - lag de tiempo a predecir
   dropout - Porcentaje expresado en decimales para la cantidad de dropout de neuronas
   optimizador - metodo de optimización 'adam'
   perdida - metrica para controlar la perdida de gradiente
   n_epocs - cantidad de iteraciones
   batch  - tamaño del lote de cada iteracion
   v_shuffle  - si shuffle sera verdadero o falso
   pred_steps - número de periodos a predecir

"""

def genera_modelo_con_preds(variable, n_registros_test, time_steps, dropout, optimizador, perdida, n_epocs, batch, v_shuffle,pred_steps):
  df_train, df_test = df.divide_train_test(variable, n_registros_test)
  df_train_escalado, df_test_escalado = df.escala_train_test(df_train, df_test)
  X_data_train, y_data_train = df.crea_datasets_train_test_preds(df_train_escalado,pred_steps,time_steps)
  X_data_test, y_data_test = df.crea_datasets_train_test_preds(df_test_escalado,pred_steps,time_steps)
  df.compila_red_reuronal(X_data_train, dropout, optimizador, perdida)
  df.ejecuta_red_reuronal(X_data_train, y_data_train, n_epocs, batch, v_shuffle)
  df.grafica_de_perdida()
  y_predicho = df.prediccion(X_data_test)
  y_test_orig, y_pred_orig, y_train_orig = df.valores_originales(y_data_test, y_predicho, y_data_train)
  df.grafica_predicciones(y_test_orig, y_pred_orig, y_train_orig)
  return df.evaluacion_modelo(y_test_orig, y_pred_orig)


